In [1]:
%pip install backtesting

     -------------------------------------- 175.5/175.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173806 sha256=5b022d402655fc1a96c1af58e123b01bd08d45bb6dd8cb359e44b12ac351084f
  Stored in directory: c:\users\tyler\appdata\local\pip\cache\wheels\3f\7c\24\f8816cdb5359accfe50ebbb023baf41e98592f11528ed26ce6
Successfully built backtesting
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
# 
# List of file names
files = ['w1Jan.csv', 'w2Jan.csv', 'w3Jan.csv','w4Jan.csv','goldRush.csv']

# Create an empty list to store the dataframes
df_list = []

# Loop through the list of files and read each file into a dataframe
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)

# Concatenate the dataframes in the list
data = pd.concat(df_list)

# Reset the index
data.reset_index(drop=True, inplace=True)
# Load the stock market data
# data = pd.read_csv('goldRush.csv')

def calculate_atr(df, period):
    true_range = np.maximum(df['High'] - df['Low'], df['High'] - df['Close'].shift(1), df['Close'].shift(1) - df['Low'])
    atr = true_range.rolling(window=period).mean()
    return atr

def atr_trailing_stop(src, atr):
    x_atr_trailing_stop = 0.0
    # n_loss = atr * 1
    result = []
    src=src.values
    atr=atr.values
    for i in range(len(src)):
        n_loss = atr[i]
        if i == 0:
            result.append(x_atr_trailing_stop)
        else:
            if src[i] > src[i-1] and src[i-1] > x_atr_trailing_stop:
                x_atr_trailing_stop = max(x_atr_trailing_stop, src[i] - n_loss)
            elif src[i] < src[i-1] and src[i-1] < x_atr_trailing_stop:
                x_atr_trailing_stop = min(x_atr_trailing_stop, src[i] + n_loss)
            elif src[i] > src[i-1]:
                x_atr_trailing_stop = src[i] - n_loss
            else:
                x_atr_trailing_stop = src[i] + n_loss
            result.append(x_atr_trailing_stop)
    return result

def ema(close_price):
    ema = close_price[0]
    ema_values = [ema]
    for i in range(1, len(close_price)):
        ema = ema * (1 - 1/1) + close_price[i] * (1/1)
        ema_values.append(ema)
    return ema_values


# Calculate the Average True Range (ATR) indicator
atr = calculate_atr(data, 10)
data['ATR'] = atr

# Calculate the ATR Trailing Stop indicator
src = data['Close']
atr_trailing_stop = atr_trailing_stop(src, atr)
data['ATR_Trailing_Stop'] = atr_trailing_stop


ema = ema(src)
data['EMA'] = ema

def buy_sell_signals(src, xATRTrailingStop, ema):
    above_below =''
    res = []
    last_signal = 0;
    for i in range(len(src)):
        # print(i)
        if i !=0:
            
            if (src[i] > xATRTrailingStop[i] and ema[i] > xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'above'
                if last_signal!=1:
                    res.append(1)
                    last_signal = 1
                else:
                    res.append(0)
            elif (src[i] < xATRTrailingStop[i] and ema[i] < xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'below'
                if last_signal!=-1:
                    res.append(-1)
                    last_signal = -1
                else:
                    res.append(0)
            else:
                res.append(0)
                last_signal = 0
        if i==0:
            res.append(0)
        
    return res

signals = buy_sell_signals(src, atr_trailing_stop, ema)
data['signal'] = signals

In [3]:
data.tail(20)

,Datetime,Open,High,Low,Close,Adj Close,Volume,ATR,ATR_Trailing_Stop,EMA,signal
27279,2023-02-10 16:40:00-05:00,1875.000000,1875.599976,1875.000000,1875.599976,1875.599976,52,0.250037,1875.349939,1875.599976,1
27280,2023-02-10 16:41:00-05:00,1875.500000,1876.400024,1875.500000,1876.199951,1876.199951,126,0.320032,1875.879919,1876.199951,0
27281,2023-02-10 16:42:00-05:00,1876.099976,1876.199951,1876.099976,1876.199951,1876.199951,13,0.280029,1876.479980,1876.199951,0
27282,2023-02-10 16:43:00-05:00,1876.099976,1876.199951,1876.099976,1876.099976,1876.099976,10,0.250024,1876.350000,1876.099976,-1
27283,2023-02-10 16:44:00-05:00,1876.000000,1876.199951,1876.000000,1876.199951,1876.199951,5,0.250012,1875.949939,1876.199951,1
27284,2023-02-10 16:45:00-05:00,1876.099976,1876.699951,1875.800049,1876.599976,1876.599976,44,0.330005,1876.269971,1876.599976,0
27285,2023-02-10 16:46:00-05:00,1876.500000,1876.599976,1876.500000,1876.599976,1876.599976,17,0.340002,1876.939978,1876.599976,0
27286,2023-02-10 16:47:00-05:00,1876.500000,1876.500000,1876.500000,1876.500000,1876.500000,2,0.329993,1876.829993,1876.500000,-1
27287,2023-02-10 16:48:00-05:00,1876.599976,1876.599976,1876.300049,1876.300049,1876.300049,8,0.349976,1876.650024,1876.300049,0
27288,2023-02-10 16:49:00-05:00,1876.199951,1876.199951,1875.699951,1875.699951,1875.699951,37,0.369971,1876.069922,1875.699951,0


In [4]:
data.set_index('Datetime')

,Open,High,Low,Close,Adj Close,Volume,ATR,ATR_Trailing_Stop,EMA,signal
Datetime,,,,,,,,,,
2023-01-16 00:00:00-05:00,1925.199951,1925.300049,1924.900024,1925.000000,1925.000000,0,NaN,0.000000,1925.000000,0
2023-01-16 00:01:00-05:00,1924.900024,1924.900024,1924.500000,1924.599976,1924.599976,95,NaN,NaN,1924.599976,0
2023-01-16 00:02:00-05:00,1924.599976,1924.599976,1923.900024,1923.900024,1923.900024,132,NaN,NaN,1923.900024,0
2023-01-16 00:03:00-05:00,1923.900024,1924.099976,1923.800049,1923.800049,1923.800049,105,NaN,NaN,1923.800049,0
2023-01-16 00:04:00-05:00,1923.800049,1923.800049,1923.300049,1923.300049,1923.300049,71,NaN,NaN,1923.300049,0
...,...,...,...,...,...,...,...,...,...,...
2023-02-10 16:55:00-05:00,1876.099976,1876.400024,1876.000000,1876.300049,1876.300049,34,0.300012,1876.600061,1876.300049,-1
2023-02-10 16:56:00-05:00,1876.300049,1876.500000,1876.199951,1876.300049,1876.300049,15,0.320020,1876.620068,1876.300049,0
2023-02-10 16:57:00-05:00,1876.300049,1876.599976,1876.099976,1876.199951,1876.199951,58,0.370020,1876.569971,1876.199951,0


In [23]:
RiskRewardRatio = 7
percentage = 0.0001

In [24]:
from backtesting import Strategy
def SIGNAL():
    return data.signal
class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==1:
            sl1 =self.data.Close[-1]*(1-percentage)
            tp1 = self.data.Close[-1]*(1+percentage*RiskRewardRatio)
            # print(self.data.Close[-1],sl1,tp1)
            self.buy(sl=sl1, tp=tp1)
            
        elif self.signal1==-1:
            sl1 =self.data.Close[-1]*(1+percentage)
            tp1 = self.data.Close[-1]*(1-percentage*RiskRewardRatio)
            self.sell(sl=sl1, tp=tp1)

In [25]:

data_new = data.copy()                             # Create copy of pandas DataFrame
data_new['Datetime'] = pd.to_datetime(data_new['Datetime'])

In [26]:
data_new = data_new.set_index(pd.DatetimeIndex(data_new['Datetime']))
data_new

,Datetime,Open,High,Low,Close,Adj Close,Volume,ATR,ATR_Trailing_Stop,EMA,signal
Datetime,,,,,,,,,,,
2023-01-16 00:00:00-05:00,2023-01-16 00:00:00-05:00,1925.199951,1925.300049,1924.900024,1925.000000,1925.000000,0,NaN,0.000000,1925.000000,0
2023-01-16 00:01:00-05:00,2023-01-16 00:01:00-05:00,1924.900024,1924.900024,1924.500000,1924.599976,1924.599976,95,NaN,NaN,1924.599976,0
2023-01-16 00:02:00-05:00,2023-01-16 00:02:00-05:00,1924.599976,1924.599976,1923.900024,1923.900024,1923.900024,132,NaN,NaN,1923.900024,0
2023-01-16 00:03:00-05:00,2023-01-16 00:03:00-05:00,1923.900024,1924.099976,1923.800049,1923.800049,1923.800049,105,NaN,NaN,1923.800049,0
2023-01-16 00:04:00-05:00,2023-01-16 00:04:00-05:00,1923.800049,1923.800049,1923.300049,1923.300049,1923.300049,71,NaN,NaN,1923.300049,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-10 16:55:00-05:00,2023-02-10 16:55:00-05:00,1876.099976,1876.400024,1876.000000,1876.300049,1876.300049,34,0.300012,1876.600061,1876.300049,-1
2023-02-10 16:56:00-05:00,2023-02-10 16:56:00-05:00,1876.300049,1876.500000,1876.199951,1876.300049,1876.300049,15,0.320020,1876.620068,1876.300049,0
2023-02-10 16:57:00-05:00,2023-02-10 16:57:00-05:00,1876.300049,1876.599976,1876.099976,1876.199951,1876.199951,58,0.370020,1876.569971,1876.199951,0


In [27]:
from backtesting import Backtest

bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)
stat = bt.run()
print(stat)
# bt.plot()

C:\Users\tyler\AppData\Local\Temp\ipykernel_20164\2153969534.py:3: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(data_new, MyCandlesStrat, cash=1000000, commission=-0.00)


Start                     2023-01-16 00:00...
End                       2023-02-10 16:59...
Duration                     25 days 16:59:00
Exposure Time [%]                   72.303015
Equity Final [$]                 797579.58452
Equity Peak [$]                1021891.358134
Return [%]                         -20.242042
Buy & Hold Return [%]               -2.524674
Return (Ann.) [%]                  -96.697049
Volatility (Ann.) [%]                1.082441
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.564062
Avg. Drawdown [%]                   -0.328588
Max. Drawdown Duration       14 days 18:13:00
Avg. Drawdown Duration        0 days 06:59:00
# Trades                                 7371
Win Rate [%]                        13.783747
Best Trade [%]                       0.739567
Worst Trade [%]                      -2.68926
Avg. Trade [%]                    

In [28]:
print(stat)

Start                     2023-01-16 00:00...
End                       2023-02-10 16:59...
Duration                     25 days 16:59:00
Exposure Time [%]                   72.303015
Equity Final [$]                 797579.58452
Equity Peak [$]                1021891.358134
Return [%]                         -20.242042
Buy & Hold Return [%]               -2.524674
Return (Ann.) [%]                  -96.697049
Volatility (Ann.) [%]                1.082441
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.564062
Avg. Drawdown [%]                   -0.328588
Max. Drawdown Duration       14 days 18:13:00
Avg. Drawdown Duration        0 days 06:59:00
# Trades                                 7371
Win Rate [%]                        13.783747
Best Trade [%]                       0.739567
Worst Trade [%]                      -2.68926
Avg. Trade [%]                    

In [16]:
import matplotlib.pyplot as plt
bt.plot()

C:\Users\tyler\anaconda3\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '5T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of values (4) does not match length of index (2)